In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion
from gammapy.data import DataStore
from gammapy.maps import Map, MapAxis, WcsGeom, WcsNDMap
from gammapy.cube import MapMaker, MapFit, PSFKernel
from gammapy.cube.models import SkyModel
from gammapy.spectrum.models import PowerLaw, ExponentialCutoffPowerLaw
from gammapy.image.models import SkyGaussian, SkyPointSource
from gammapy.detect import TSMapEstimator
from gammapy.scripts import SpectrumAnalysisIACT
from pyV2DL3.generateObsHduIndex import create_obs_hdu_index_file
import astropy.units as u
from gammapy.background import (
    ReflectedRegionsBackgroundEstimator,
    ring_background_estimate,
)
from gammapy.spectrum import     SpectrumFit,SpectrumExtraction,SpectrumResult,CrabSpectrum
from gammapy.spectrum import SpectrumEnergyGroupMaker
from gammapy.spectrum import FluxPointEstimator
from gammapy.irf import make_mean_psf

In [ ]:
def spectral_analysis(datastore,on_region,exclusion_mask,point_like=False,plots=False,nbins=81):
    # Bakg estimate and extract spectrum
    observations=datastore.get_observations(data_store.obs_table['OBS_ID'])
    bkg_estimate = ReflectedRegionsBackgroundEstimator(
    observations=observations, on_region=on_region, exclusion_mask=exclusion_mask
    )
    bkg_estimate.run()
    energy_bins = np.logspace(np.log10(0.01), np.log10(100), nbins) * u.TeV
    extract = SpectrumExtraction(
        observations=observations,
        bkg_estimate=bkg_estimate.result,
        e_true=energy_bins,
        e_reco=energy_bins,
        containment_correction=(not point_like),
    )
    extract.run()
    if(plots):
        extract.spectrum_observations[0].peek()
    # Fit spectrum 
    model = PowerLaw(
    index=2, amplitude=2e-11 * u.Unit("m-2 s-1 TeV-1"), reference=1 * u.TeV
    )

    joint_fit = SpectrumFit(obs_list=extract.spectrum_observations, model=model)
    joint_fit.run()
    joint_result = joint_fit.result
    if(plots):
        ax0, ax1 = joint_result[0].plot(figsize=(8, 8))
        ax0.set_ylim(0, 20)
        print(joint_result[0])
    stacked_obs = extract.spectrum_observations.stack()
    e_min, e_max = stacked_obs.lo_threshold.to_value("TeV"), 30
    ebounds = np.logspace(np.log10(e_min), np.log10(e_max), 10) * u.TeV

    seg = SpectrumEnergyGroupMaker(obs=stacked_obs)
    seg.compute_groups_fixed(ebounds=ebounds)
    if(plots):
        print(seg.groups)
    fpe = FluxPointEstimator(
        obs=stacked_obs, groups=seg.groups, model=joint_result[0].model
    )
    flux_points = fpe.run()
    flux_points.table["is_ul"] = flux_points.table["ts"] < 4
    spectrum_result = SpectrumResult(
    points=flux_points, model=joint_result[0].model
    )
    return extract,flux_points,spectrum_result

# Define on region

In [ ]:
pos_crab = SkyCoord(83.633, 22.014, unit="deg")
on_region = CircleSkyRegion(center=pos_crab, radius=0.1*u.deg)
exclusion_mask = Map.read('/Users/tony/Desktop/AtmTransparency/DL3_cross_calibration/joint-crab/results/maps/exclusion_mask.fits.gz')

# Draw PSF Kernel

In [ ]:
create_obs_hdu_index_file(["./full_enclosure/89395.fits"])
data_store = DataStore.from_dir('./full_enclosure/')
observations = data_store.get_observations(data_store.obs_table["OBS_ID"])
obs = observations[0]
table_psf = obs.psf.to_energy_dependent_table_psf(u.deg*0.5)
energy_axis = MapAxis.from_edges(
    np.logspace(0, 1.0, 5), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(83.633, 22.014),
    binsz=0.02,
    width=(5, 5),
    coordsys="CEL",
    proj="TAN",
    axes=[energy_axis],
)
psf_kernel = PSFKernel.from_table_psf(table_psf, geom, max_radius="0.3 deg")
psf_kernel_array = psf_kernel.psf_kernel_map.sum_over_axes().data
plt.figure(figsize=(20,20))
psf_kernel.psf_kernel_map.slice_by_idx({'energy': 0}).plot()

In [ ]:
psf = observations[0].psf
psf.plot_psf_vs_rad()
plt.figure()
psf.plot_containment(show_safe_energy=True)
plt.figure()
psf.plot_containment_vs_energy()

## The binning for the histogram is too fine
## Might need to merge bins --> How to do this in V2DL3 ?

# Spectral Analysis -- Full Enclosure

In [ ]:
create_obs_hdu_index_file(["./full_enclosure/89395.fits"],index_file_dir='./full_enclosure//')
data_store = DataStore.from_dir('./full_enclosure/')
extract_fe,flux_points_fe,spectrum_result_fe = spectral_analysis(data_store,on_region,exclusion_mask,point_like=False,plots=True)

In [ ]:
crab_hess_pl = CrabSpectrum('hess_pl')
ax0, ax1 = spectrum_result_fe.plot(
    energy_range=[0.2,45]*u.TeV,
    energy_power=0,
    flux_unit="TeV-1 m-2 s-1",
    fig_kwargs=dict(figsize=(8, 8)),
)
crab_hess_pl.model.plot([0.2, 10] * u.TeV,ax=ax0,energy_power=0,flux_unit="TeV-1 m-2 s-1")
ax0.set_xlim(0.4, 50)


## Point Like

In [ ]:
create_obs_hdu_index_file(["./point_like//89395.fits"],index_file_dir='./point_like//')
data_store = DataStore.from_dir('./point_like/')
extract_pl,flux_points_pl,spectrum_result_pl = spectral_analysis(data_store,on_region,exclusion_mask,point_like=True,plots=True)

In [ ]:
crab_hess_pl = CrabSpectrum('hess_pl')
ax0, ax1 = spectrum_result_pl.plot(
    energy_range=[0.2,45]*u.TeV,
    energy_power=0,
    flux_unit="TeV-1 m-2 s-1",
    fig_kwargs=dict(figsize=(8, 8)),
)
crab_hess_pl.model.plot([0.2, 10] * u.TeV,ax=ax0,energy_power=0,flux_unit="TeV-1 m-2 s-1")
ax0.set_xlim(0.4, 50)

# Comparison

In [ ]:
plt.figure(figsize=(15,10))
flux_points_fe.plot(flux_unit='TeV-1 m-2 s-1',color='darkorange',label='Full Enclosure ')
flux_points_pl.plot(flux_unit='TeV-1 m-2 s-1',color='darkblue',label='Point Like')
crab_hess_pl.model.plot([0.2, 40] * u.TeV,energy_power=0,flux_unit="TeV-1 m-2 s-1",label='HESS PL')
plt.legend(loc='best')

In [ ]:
plt.figure(figsize=(15,10))
crab_hess_pl.model.plot([0.2, 40] * u.TeV,energy_power=0,flux_unit="TeV-1 m-2 s-1",label='HESS PL',color='darkgreen',linewidth=3)
spectrum_result_fe.model.plot([0.2, 40] * u.TeV,energy_power=0,flux_unit="TeV-1 m-2 s-1",
                              label='Full Enclosure',color='darkorange',linewidth=3)
spectrum_result_pl.model.plot([0.2, 40] * u.TeV,energy_power=0,flux_unit="TeV-1 m-2 s-1",
                              label='Point Like',color='darkblue',linewidth=3)
plt.legend(loc='best')

## Compare Effective Area

In [ ]:
fe_spo = extract_fe.spectrum_observations[0]
pl_spo = extract_pl.spectrum_observations[0]
plt.figure(figsize=(15,10))
fe_spo.aeff.plot(label='Full Enclosure',show_energy=(fe_spo.hi_threshold, fe_spo.lo_threshold))
ax = plt.gca()
ax.set_xlim([0.1,100])

pl_spo.aeff.plot(ax=ax,label='Point Like')
plt.legend(loc='best')

# Integral Flux -- Point Like

In [ ]:
int_crab_flux  = crab_hess_pl.model.integral(1.0*u.TeV,100*u.TeV).to('m-2 s-1')

In [ ]:
integral,err = spectrum_result_pl.model.integral_error(1.0*u.TeV,100*u.TeV)
print('Integral Flux (>1TeV):{:.2e}'.format(integral))
print('Integral Flux Error (>1TeV):{:.2e}'.format(err))
print('{:.1f}'.format(spectrum_result_pl.model.integral(1.0*u.TeV,100*u.TeV)/int_crab_flux*100),'% Crab')

# Integral Flux -- Full Enclosure

In [ ]:
spectrum_result_fe.model.integral_error?

In [ ]:
integral,err = spectrum_result_fe.model.integral_error(1.0*u.TeV,100*u.TeV)
print('Integral Flux (>1TeV):{:.2e}'.format(integral))
print('Integral Flux Error (>1TeV):{:.2e}'.format(err))
print('{:.1f}'.format(spectrum_result_fe.model.integral(1.0*u.TeV,100*u.TeV)/int_crab_flux*100),'% Crab')

## VEGAS' result for the same run
## Integral flux (1 TeV) $1.26\times 10^{-7} \pm 5.60\times10^{-8} m^{-2}s^{-1}$

# Test using HESS' DL3 file

In [ ]:
data_store = DataStore.from_dir(
    "/Users/tony/Desktop/AtmTransparency/DL3_cross_calibration/joint-crab/data/hess/"
)
mask = data_store.obs_table["TARGET_NAME"] == "Crab"
obs_table = data_store.obs_table[mask]
observations = data_store.get_observations(obs_table["OBS_ID"])

In [ ]:
data_store = DataStore.from_dir(
    "/Users/tony/Desktop/AtmTransparency/DL3_cross_calibration/joint-crab/data/hess/"
)
mask = data_store.obs_table["TARGET_NAME"] == "Crab"
obs_table = data_store.obs_table[mask]
observations = data_store.get_observations(obs_table["OBS_ID"])
psf = observations[0].psf
psf.plot_psf_vs_rad()
plt.figure()
psf.plot_containment(show_safe_energy=True)
plt.figure()
psf.plot_containment_vs_energy()

In [ ]:
pos_crab = SkyCoord(83.633, 22.014, unit="deg")

In [ ]:
energy_axis = MapAxis.from_edges(
    np.logspace(0, 1.0, 5), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(83.633, 22.014),
    binsz=0.02,
    width=(5, 5),
    coordsys="CEL",
    proj="TAN",
    axes=[energy_axis],
)

In [ ]:
from gammapy.irf import make_mean_psf
table_psf = make_mean_psf(observations, pos_crab)
psf_kernel = PSFKernel.from_table_psf(table_psf, geom, max_radius="0.3 deg")
psf_kernel_array = psf_kernel.psf_kernel_map.sum_over_axes().data
plt.figure(figsize=(20,20))
psf_kernel.psf_kernel_map.slice_by_idx({'energy': 0}).plot()
#plt.imshow(psf_kernel_array)